# Report 3

## Basic Graph Algorithms

#### Marcin Kapiszewski 156048
#### Adam Tomys 156057

Group 2

### Compared diraph representations:

1. Incidence matrix:
    - Matrix consists of vertices (rows), arcs (columns), and values(cells)
    - Values:
        - 1 means the head of the arc
        - 0 means that the arc is not connected to that node
        - -1 means the tail of the arc
2. Adjacency matrix:
    - Matrix consists of starting vertices (rows), ending vertices (columns), and values(cells)
    - Values:
        - 1 means that the given arc from tail to head exists
        - 0 means that the given arc from tail to head does not exist
        - -1 means that there is an arc from ending vertice to starting vertice (this value is not required)
3. Arc list:
    - The list contains entires containing arcs
    - arc is represented as a tuple containing the tail and the head
4. Adjacency list:
    - It consists of one node for each vertice
    - Each node contains all of the successors of the vertice
    - Its possible to store all precessors too but as an additional list
5. Forward star (in our case uses AVL tree):
    - It is the same as the previos one, but instead of storing successors in a list it uses an AVL tree (in our case) or hashtable

### Implementation Difficulties:
NO difficulties so far

### DAG Generation:
TODO you better understood it and its not written anyway

In [1]:
%load_ext autoreload
%autoreload 2

### COMPARE GRAPH REPRESENTATION DEPENDING ON DAG SATURATION AND DENSITY
TODO Probably should be done mathematically not by program

### Compared topological sorts:

1. Inspired by Kahn algorithm:
    - First we count for each node the number of predecessors
    - We initalize an output list, a number of visited nodes as 0, and a queue containing all vertices that do not have any predecessors
    - While our queue is not empty:
        - Increment visited nodes by one
        - Remove a vertice from queue and add it to the output
        - For each successor of dequeued node:
            - Decrease their number of predecessors by one
            - If the number is now 0 add it to the queue
    - If the number of visisted nodes is diffrent than the number of all nodes it is not possible to sort this graph and an error or message should be returned
    - Otherwise return out output list
2. Making use of DFS (graph coloring):
    - Our code used recursion for this
    - First we initialize a stack as our output and a set for visited nodes
    - Now we iterate over every node:
        - If it was not visited already we call our helper function dfs
    - At the end we return out stack as a list (stack was used for fast pushing new values on the left)

    - Our helper dfs function:
        - It takes the graph, the current node, the set of visited nodes, and the output stack as arguments
        - It adds the node to the visited set
        - For each successors of the node if that successor was not visited it calls dfs again
        - At the end the current node is pushed to the left of the stack

In [2]:
from collections import defaultdict

from generate_DAG import generateDAG

from kahn import kahn_sort
from dfs_sort import dfs_sort

from graph_representations.adjacencyList import AdjacencyList
from graph_representations.adjacencyMatrix import AdjacencyMatrix
from graph_representations.arcList import ArcList
from graph_representations.forwardStar import ForwardStar
from graph_representations.incidenceMatrix import IncidenceMatrix
from graph_representations.digraphRepresentation import Digraph

from evaluate_program import measureTime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
sortAlgorithms = [kahn_sort, dfs_sort]
graphRepresentations = [IncidenceMatrix, AdjacencyMatrix, ArcList, AdjacencyList, ForwardStar]
nodesNumbers = [100, 200, 300, 400, 500, 750, 1000, 1250, 1500, 2000, 2500, 3000, 4000, 5000]

In [4]:
def inicializaceGraph(graph: Digraph, DAG):
    for node in range(len(DAG)):
        graph.addNode(node)
    for startNode in range(len(DAG)):
        for endNode in range(len(DAG)):
            if DAG[startNode][endNode] == 1:
                graph.addEdge(startNode, endNode)

In [5]:
def evaluateSortAlgorithm(sortAlgorithm):
    graphRepresentation: Digraph
    times = {} # times.nodesNumber.graphRepresentation
    for nodesNumber in nodesNumbers:
        print(f"{nodesNumber}:")
        times[nodesNumber] = {}
        DAG = generateDAG(nodesNumber)
        for graphRepresentation in graphRepresentations:
            if (graphRepresentation.__name__ == "IncidenceMatrix" and nodesNumber >= 1500) or (graphRepresentation.__name__ == "ArcList" and nodesNumber >= 2000):
                continue
            graph = graphRepresentation()
            inicializaceGraph(graph, DAG)
            times[nodesNumber][graphRepresentation.__name__] = measureTime(sortAlgorithm, [graph])
            print(f"\t{graphRepresentation.__name__} --> {times[nodesNumber][graphRepresentation.__name__]}")
    return times

In [6]:
times = {}
for sortAlgorithm in sortAlgorithms:
    times[sortAlgorithm.__name__] = evaluateSortAlgorithm(sortAlgorithm)

100:
	IncidenceMatrix --> 0.07241463661193848
	AdjacencyMatrix --> 0.007050991058349609
	ArcList --> 0.017047643661499023
	AdjacencyList --> 0.0
	ForwardStar --> 0.0005192756652832031
200:
	IncidenceMatrix --> 0.5595002174377441
	AdjacencyMatrix --> 0.009897470474243164
	ArcList --> 0.12316250801086426
	AdjacencyList --> 0.009442806243896484
	ForwardStar --> 0.013160228729248047
300:
	IncidenceMatrix --> 1.8069813251495361
	AdjacencyMatrix --> 0.017364501953125
	ArcList --> 0.40325307846069336
	AdjacencyList --> 0.008790969848632812
	ForwardStar --> 0.03179764747619629
400:
	IncidenceMatrix --> 3.8091235160827637
	AdjacencyMatrix --> 0.032140493392944336
	ArcList --> 0.8691544532775879
	AdjacencyList --> 0.016213178634643555
	ForwardStar --> 0.05322575569152832
500:
	IncidenceMatrix --> 7.29220724105835
	AdjacencyMatrix --> 0.0441286563873291
	ArcList --> 1.8299329280853271
	AdjacencyList --> 0.02230978012084961
	ForwardStar --> 0.08801603317260742
750:
	IncidenceMatrix --> 26.52824091

In [26]:
def compareSortTimesForGraphSizes(times, title):
    titles = [f"nodes: {nodesNumber}  edges:{((nodesNumber-1)*nodesNumber)//2}" for nodesNumber in  nodesNumbers]
    fig = make_subplots(rows=((len(times) - 1)//2 + 1), cols=2,subplot_titles=titles)

    i=0
    for nodesNumber in nodesNumbers:
        d = times[nodesNumber]
        fig.add_trace(go.Bar(x=list(d.keys()), y=list(d.values()), text=[round(x,3) for x in d.values()],
                            textposition="auto", name=""),
                        row=i//2+1, col=i%2+1)
        fig.update_yaxes(title_text="time [s]", row=i//2+1, col=i%2+1)
        i+=1

    fig.update_layout(height=1600, width=1200,
                    title_text=title)

    fig.show()

In [27]:
def compareSortTimesForGraphRepresentation(times, title):
    fig = make_subplots(rows=3, cols=2, subplot_titles=[representation.__name__ for representation in graphRepresentations])

    i=0
    for graphRepresentation in graphRepresentations:
        d = defaultdict(lambda : {})
        for nodesNumber, time in times.items():
            # edgesNumber = ((nodesNumber-1)*nodesNumber)/2
            if graphRepresentation.__name__ in time.keys():
                d[nodesNumber] = time[graphRepresentation.__name__]
        fig.add_trace(go.Scatter(x=list(d.keys()), y=list(d.values()), text=[round(x,3) for x in d.values()]),
                        row=i//2+1, col=i%2+1)
        fig.update_yaxes(title_text="time [s]", row=i//2+1, col=i%2+1)
        i+=1

    fig.update_layout(height=800, width=1200,
                    title_text=title)

    fig.show()

In [28]:
compareSortTimesForGraphRepresentation(times["kahn_sort"], "kahn_sort")

In [29]:
compareSortTimesForGraphSizes(times["kahn_sort"], "kahn_sort")

In [31]:
compareSortTimesForGraphRepresentation(times["dfs_sort"], "dfs_sort")

In [30]:
compareSortTimesForGraphSizes(times["dfs_sort"], "dfs_sort")

### Conclusions:

TODO first rest of report needed
    
